## 🧹 Importing necessary libraries

## Structured Approach to Advanced Predictions 📘📐
We adhere to a systematic and analytical approach:
1. **Data Preparation**: Stringent preprocessing for a robust modeling foundation.
2. **Feature Engineering**: Deliberate crafting of features that reflect the dynamics of the market.
3. **Purging Cross-Validation**: Implementing an innovative CV strategy that honors the temporal aspect of financial data.
4. **Model Training**: Concentrating on models that excel in precision and generalizability.
5. **Prediction & Strategy**: Crafting a well-thought-out plan for prediction and competition submission.

In [1]:
import gc  # Garbage collection for memory management
import os  # Operating system-related functions
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling

# 📦 Importing machine learning libraries
import joblib  # For saving and loading models
import lightgbm as lgb  # LightGBM gradient boosting framework
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques
from catboost import CatBoostRegressor, Pool
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm

from tqdm.notebook import tqdm  # Progress bar
import seaborn as sns
import matplotlib.pyplot as plt
import numba

# 🤐 Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# 📊 Define flags and variables
is_offline = False  # Flag for online/offline mode
is_train = False  # Flag for training mode
is_infer = True  # Flag for inference mode
max_lookback = np.nan  # Maximum lookback (not specified)
split_day = 435  # Split day for time series data

DATE_NUM = 481
SEC_NUM = 55
STOCK_NUM = 200
LAST_SEC = 540

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


## 📊 Data Loading and Preprocessing 📊






In [2]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df_shape = df.shape

### Missed Stock_ids

In [3]:
def fill_missed_rows(df, mode='per_date'):
    assert 'row_id' in df.columns
    row_ids = set(df['row_id'].values)
    all_row_ids = set()
    if mode == 'per_second':  # ensures only all stocks per second
        seconds = df['seconds_in_bucket'].unique()
    elif mode == 'per_date':  # ensures all stocks and seconds per date
        seconds = range(0, 541, 10)
    # find missed rows
    for d in df['date_id'].astype(int).unique():
        for s in range(STOCK_NUM):
            for t in seconds:
                all_row_ids.add(f'{d}_{t}_{s}')
    rows_missed = list(all_row_ids - row_ids)
    
    # dataframe with messed rows
    df_missed = pd.DataFrame({'row_id':rows_missed})
    df_missed['date_id'] = [int(float(d)) for d, t, s in [row.split('_') for row in rows_missed]]
    df_missed['seconds_in_bucket'] = [int(float(t)) for d, t, s in [row.split('_') for row in rows_missed]]
    df_missed['stock_id'] = [int(float(s)) for d, t, s in [row.split('_') for row in rows_missed]]

    df = pd.concat([df, df_missed])
    df = df.sort_values(by=['date_id','seconds_in_bucket','stock_id']).reset_index(drop=True)
    df.reset_index(drop=True)
    return df

## 🚀 Memory Optimization Function with Data Type Conversion 🧹

In [4]:
# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    if verbose:
        start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)

    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")
        
    return df


 ## 🏎️Parallel Triplet Imbalance Calculation with Numba

In [5]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)
    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features


## Lag-Depended Functions

In [6]:
def add_lag_lastsec(df, col_name, lag, func):
    ''' ASSUMPTION: dates are consistent!!! '''
    col_lag = [f'{col}_lastsec_lag_{lag}' for col in col_name]
    date_ids = df['date_id']
    min_date = date_ids.min()
    max_date = date_ids.max()
    if max_date - min_date < lag:
        df[col_lag] = np.nan
        return df
    for group_id, values in df.groupby(['date_id'])[col_name]:
        curr_date = int(np.array([group_id]).flatten()[0])
        lag_values = df[(df['date_id'] == curr_date - lag) & (df['seconds_in_bucket'] == LAST_SEC)][col_name]
        if not lag_values.empty:
            df.loc[values.index, col_lag] = np.tile(lag_values, (len(values)//len(lag_values),1))
        else:
            df.loc[values.index, col_lag] = np.nan
    return df

def add_lag_constsec(df, col_name, lag, func):
    ''' ASSUMPTION: dates are consistent!!! '''
    if 'lag' in func or 'shift' in func:
        col_lag = [f'{col}_constsec_lag_{lag}' for col in col_name]
        lag_values = df[col_name].shift(lag * STOCK_NUM * SEC_NUM)
        df[col_lag] = lag_values
    if 'pct' in func or 'ret' in func:
        col_ret = [f'{col}_constsec_ret_{lag}' for col in col_name]
        ret_values = df[col_name].pct_change(lag * STOCK_NUM * SEC_NUM)
        df[col_ret] = ret_values
    if 'diff' in func :
        col_diff = [f'{col}_constsec_diff_{lag}' for col in col_name]
        diff_values = df[col_name].diff(lag * STOCK_NUM * SEC_NUM)
        df[col_diff] = diff_values
    return df

def add_lag_nolimit(df, col_name, lag, func):
    ''' ASSUMPTION: dates are consistent!!! '''
    if 'lag' in func or 'shift' in func:
        col_lag = [f'{col}_nolimit_lag_{lag}' for col in col_name]
        lag_values = df[col_name].shift(lag * STOCK_NUM)
        df[col_lag] = lag_values
    if 'pct' in func or 'ret' in func:
        col_ret = [f'{col}_nolimit_ret_{lag}' for col in col_name]
        ret_values = df[col_name].pct_change(lag * STOCK_NUM)
        df[col_ret] = ret_values
    if 'diff' in func :
        col_diff = [f'{col}_nolimit_diff_{lag}' for col in col_name]
        diff_values = df[col_name].diff(lag * STOCK_NUM)
        df[col_diff] = diff_values
    return df

def add_lag_intraday(df, col_name, lag, func):
    ''' ASSUMPTION: dates are consistent!!! '''
    if 'lag' in func or 'shift' in func:
        col_lag = [f'{col}_intraday_lag_{lag}' for col in col_name]
        lag_values = df[col_name].shift(lag * STOCK_NUM)
        df[col_lag] = lag_values
        df.loc[df['seconds_in_bucket'] < lag*10, col_lag] = np.nan
    if 'pct' in func or 'ret' in func:
        col_ret = [f'{col}_intraday_ret_{lag}' for col in col_name]
        ret_values = df[col_name].pct_change(lag * STOCK_NUM)
        df[col_ret] = ret_values
        df.loc[df['seconds_in_bucket'] < lag*10, col_ret] = np.nan
    if 'diff' in func :
        col_diff = [f'{col}_intraday_diff_{lag}' for col in col_name]
        diff_values = df[col_name].diff(lag * STOCK_NUM)
        df[col_diff] = diff_values
        df.loc[df['seconds_in_bucket'] < lag*10, col_diff] = np.nan
    return df

def generate_lag_features(df, col_name, lags, func, lag_type='nolimit'):
    """
    Adds Lag-depended features.
    
    Parameters:
    df (pandas.DataFrame): input DataFrame.
    col_name (str): column for lag.
    lags (list): lag values for funcs.
    func (str): lag-dependent function: lag, ret, diff, roll
    lag_type (str): type of information access method: intraday, constsec, nolimit, lastsec (lag_type=lastsec is compatible with func=lag only!).
    
    Returns:
    pandas.DataFrame: result DataFrame
    """
    lag_type_funcs = {'intraday': add_lag_intraday, 
                     'constsec': add_lag_constsec, 
                     'nolimit': add_lag_nolimit,
                     'lastsec': add_lag_lastsec}
    lag_type_func = lag_type_funcs[lag_type]
       
    for lag in lags:
        df = lag_type_func(df, col_name, lag, func)
    return df

## Mean by Day / Second

In [7]:
@numba.njit("float32[:](float32[:], int64)", parallel=True, nogil=True)
def mean_by_period_jit(x, w):
    y = np.empty_like(x)
    iters = len(x) // w
    for i in numba.prange(iters):
        y[i*w: (i+1)*w] = np.nanmean(x[i*w: (i+1)*w])
    return y

def mean_by_period(df, col_name, period='sec'):
    w = STOCK_NUM
    if period == 'day':
        w = STOCK_NUM * SEC_NUM
    
    for col in col_name:
        mean_col = f'{col}_mean'
        x = df[col].values
        df[mean_col] = mean_by_period_jit(x, w)
        
    return df

def mean_and_diff_by_period(df, col_name, period='sec'):
    w = STOCK_NUM
    if period == 'day':
        w = STOCK_NUM * SEC_NUM
    
    for col in col_name:
        mean_col = f'{col}_mean'
        mean_diff_col = f'{col}_mean_diff'
        x = df[col].values
        df[mean_col] = mean_by_period_jit(x, w)
        df[mean_diff_col] = df[col] - df[mean_col]
        
    return df

## 📊 Feature Generation Functions 📊






In [8]:
# 📊 Function to generate imbalance features
def imbalance_features(df, log=False):
    start_time = time.time()
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size_signed"]

    # V1: syntetic
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff(periods=1)  # df = generate_lag_features(df, col_name='price_spread', lags=[1], func=['diff'], lag_type='intraday')
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
#     df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df['price_diff'] = df['reference_price'] - df['wap']
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    df['reference_price_wap_imb'] = df.eval("(reference_price-wap) / (reference_price+wap)")
    df['reference_price_bid_price_imb'] = df.eval("(reference_price-bid_price) / (reference_price+bid_price)")   
    df['far_price_near_price_imb'] = df.eval("(far_price - near_price) / (far_price + near_price)")   
    df['far_price_over_near_price'] = df["far_price"].div(df["near_price"])
    df['far_price_minus_near_price'] = df["far_price"] - df["near_price"]
    
    if log:
        print('Created: V1 |', time.time() - start_time)
        start_time = time.time()
    
    # V2: imbalances
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    if log:
        print('Created: V2 |', (time.time() - start_time))   
        start_time = time.time()
    
    # V3: lag features
    col_names = ["reference_price", "ask_price", "bid_price", "wap"] + sizes
    df = generate_lag_features(df, col_name=col_names, lags=[1, 2, 5, 10, 15], func=['lag', 'ret'], lag_type='nolimit')
    df = generate_lag_features(df, col_name=prices, lags=[1, 2, 5, 10, 15], func=['diff'], lag_type='nolimit')
    if log:
        print('Created: V3 |', time.time() - start_time)
        start_time = time.time()
        
    # V4: lag target
    df = generate_lag_features(df, col_name=['target'], lags=[1, 2, 3, 4], func=['lag'], lag_type='constsec')
    df = generate_lag_features(df, col_name=['target'], lags=[1, 2, 3, 4], func=['lag'], lag_type='lastsec')
    if log:
        print('Created: V4 |', time.time() - start_time)
        start_time = time.time()
        
    # V5: mean features + meandiffs
    price_mean_col_names = ["reference_price", "ask_price", "bid_price", "wap",
                            "reference_price_nolimit_ret_1", "ask_price_nolimit_ret_1", "bid_price_nolimit_ret_1", "wap_nolimit_ret_1",
                            "reference_price_nolimit_diff_1", "ask_price_nolimit_diff_1", "bid_price_nolimit_diff_1", "wap_nolimit_diff_1",]
    size_mean_col_names = ["matched_size", "bid_size", "ask_size", "imbalance_size_signed"]
#                            "matched_size_nolimit_ret_1", "bid_size_nolimit_ret_1", "ask_size_nolimit_ret_1", "imbalance_size_signed_nolimit_ret_1",
#                            "matched_size_nolimit_diff_1", "bid_size_nolimit_diff_1", "ask_size_nolimit_diff_1", "imbalance_size_signed_nolimit_diff_1"]
    df = mean_and_diff_by_period(df, col_name=price_mean_col_names, period='sec')
    df = mean_and_diff_by_period(df, col_name=size_mean_col_names, period='sec')
    
    reduced_prices = ["reference_price", "ask_price", "bid_price", "wap"]
    sma_col_names = ["sma_10_reference_price", "sma_10_ask_price", "sma_10_bid_price", "sma_10_wap"]
    sma_diff_col_names = ["sma_10_reference_price_currdiff", "sma_10_ask_price_currdiff", "sma_10_bid_price_currdiff", "sma_10_wap_currdiff"]
    for i in range(len(reduced_prices)):
        df[sma_diff_col_names[i]] = df[reduced_prices[i]] - df[sma_col_names[i]]
    if log:
        print('Created: V5 |', time.time() - start_time)

    return df.replace([np.inf, -np.inf], 0)

## Technical Analysis Indicators

### 0. SMA & EMA

In [9]:
@numba.njit("float32[:](float32[:], int64)", nogil=True)
def sma1d(x, w):
    y = np.empty_like(x)
    asum = 0.0
    for i in range(w):
        asum += x[i]
        y[i] = asum / w
    for i in range(w, len(x)):
        asum += x[i] - x[i - w]
        y[i] = asum / w
    return y

def sma2d(x, w):  # 144 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
    y = np.empty_like(x)
    for i in range(x.shape[1]):
        y[:, i] = sma1d(x[:, i], w)
    return y

@numba.njit("float32[:,:](float32[:,:], int64)", nogil=True, parallel=True)
def sma2d_jit(x, w):  # 203 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    y = np.empty_like(x)
    for i in numba.prange(x.shape[1]):
        y[:, i] = sma1d(x[:, i], w)
    return y

In [10]:
def calculate_sma_jit(data, period=14):
    init_shape = data.shape
    data = data.reshape((-1, STOCK_NUM*init_shape[1]))
    sma = sma2d_jit(data, period)
    return sma.reshape(init_shape)

In [11]:
@numba.njit("float32[:](float32[:], int64)", nogil=True)
def ema1d(x, span):
    a = 2 / (1 + span)
    y = np.empty_like(x)
    y[0] = x[0]
    for k in range(1, len(y)):
        y[k] = y[k-1]*(1 - a) + x[k]*a
    return y

def ema2d(x, span):  # 309 ms ± 485 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
    y = np.empty_like(x)
    for i in range(x.shape[1]):
        y[:, i] = ema1d(x[:, i], span)
    return y

@numba.njit("float32[:,:](float32[:,:], int64)", nogil=True, parallel=True)
def ema2d_jit(x, span):  # 238 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    y = np.empty_like(x)
    for i in numba.prange(x.shape[1]):
        y[:, i] = ema1d(x[:, i], span)
    return y

In [12]:
def calculate_ema_jit(data, period=14):
    init_shape = data.shape
    data = data.reshape((-1, STOCK_NUM*init_shape[1]))
    ema = ema2d_jit(data, period)
    return ema.reshape(init_shape)

### 1. Relative Strength Index (RSI)

$$
    \text{RSI} = 100 - \frac{100}{1 + \text{RS}}
$$

Where: 
$$
RS = (Average Gain over N days) / (Average Loss over N days)
$$

In [13]:
def calculate_rsi_jit(data, period=14):
    init_shape = data.shape
    data = data.reshape((-1, STOCK_NUM*init_shape[1]))

    delta = np.zeros_like(data)
    delta[1:] = data[1:] - data[:-1]

    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    rs = ema2d_jit(gain, period) / ema2d_jit(loss, period)
    rsi = 100 - (100 / (1 + rs))
    return rsi.reshape(init_shape)

### 2. MACD

$$
\begin{equation}
    \begin{aligned}
        \text{MACD} & = \text{EMA}(12) - \text{EMA}(26) \\
        \text{Signal Line} & = \text{EMA}(\text{MACD}, 9) \\
        \text{Histogram} & = \text{MACD} - \text{Signal Line}
    \end{aligned}
\end{equation}
$$

In [14]:
def calculate_macd_jit(data, short_window=12, long_window=26, signal_window=9):
    init_shape = data.shape
    data = data.reshape(-1, STOCK_NUM*init_shape[1])
    
    short_ema = ema2d_jit(data, short_window)
    long_ema = ema2d_jit(data, long_window)
    macd = short_ema - long_ema
    
    signal = ema2d_jit(macd, signal_window)

    histogram = macd - signal

    return macd.reshape(init_shape), signal.reshape(init_shape), histogram.reshape(init_shape)

### 3. Bollinger Bands

$$ BBands = SMA(W) \pm 2std $$

In [15]:
@numba.njit("float32(float32[:])", nogil=True)
def std_jit(x):
    x_mean = 0.0
    for i in range(len(x)):
        x_mean += x[i]
    x_mean = x_mean / len(x)
    x_delta_2 = (x - x_mean)**2
    x_delta_2_mean = 0.0
    for i in range(len(x_delta_2)):
        x_delta_2_mean += x_delta_2[i]
    x_delta_2_mean /= len(x_delta_2)
    std = np.sqrt(x_delta_2_mean)
    return std

@numba.njit("UniTuple(float32[:], 2)(float32[:], int64)", nogil=True)
def sma1d_w_std(x, w):
    y = np.empty_like(x)
    stds = np.empty_like(x)
    asum = 0.0
    for i in range(w):
        asum += x[i]
        y[i] = asum / w
        stds[i] = std_jit(x[:i+1])
    for i in range(w, len(x)):
        asum += x[i] - x[i - w]
        y[i] = asum / w
        stds[i] = std_jit(x[i-w+1: i+1])
    return y, stds

@numba.njit("UniTuple(float32[:,:], 3)(float32[:,:], int64, int64)", nogil=True, parallel=True)
def bband2d_jit(x, w, num_std_dev):  # 203 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    mid_bands = np.empty_like(x)
    upper_bands = np.empty_like(x)
    lower_bands = np.empty_like(x)
    for i in numba.prange(x.shape[1]):
        mid_bands[:, i], std_dev = sma1d_w_std(x[:, i], w)
        upper_bands[:, i] = mid_bands[:, i] + std_dev * num_std_dev
        lower_bands[:, i] = mid_bands[:, i] - std_dev * num_std_dev
    return upper_bands, mid_bands, lower_bands

In [16]:
def calculate_bband_jit(data, window=20, num_std_dev=2): #  1.54 s ± 75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)   (vs 2.19 s via Pandas)
    init_shape = data.shape
    data = data.reshape(-1, STOCK_NUM*init_shape[1])
    upper_bands, mid_bands, lower_bands = bband2d_jit(data, window, num_std_dev)
    return upper_bands.reshape(init_shape), mid_bands.reshape(init_shape), lower_bands.reshape(init_shape)

### TA Indicators Compilation

In [17]:
def generate_ta(df, lag_type='intraday'):
    """
    Generate TA features.
    
    Parameters:
    df (pandas.DataFrame): input DataFrame.
    lag_type (str): type of information access method: intraday, nolimit. //constsec is not realized
    
    Returns:
    pandas.DataFrame: result DataFrame
    """
    # Define lists of price and size-related column names
    prices = ["reference_price", "ask_price", "bid_price", "wap"]
    # sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    lag_sma = [2, 5, 10, 15]
    lag_rsi = 14
    lag_macd_short = 20
    lag_macd_long = 26
    lag_macd_sig = 9
    lag_bband = 20
    
    values = df[prices].values.astype(np.float32)
    
    # fast SMA
    for lag in lag_sma:
        col_sma = [f'sma_{lag}_{col}' for col in prices]
        sma_values = calculate_sma_jit(values, lag)
        df.loc[:, col_sma] = sma_values
    
    # fast RSI
    col_rsi = [f'rsi_{col}' for col in prices]
    rsi_values = calculate_rsi_jit(values, period=lag_rsi)
    df.loc[:, col_rsi] = rsi_values
    
    # fast MACD
    macd, signal, histogram = calculate_macd_jit(values, 
                                             short_window=lag_macd_short, 
                                             long_window=lag_macd_long, 
                                             signal_window=lag_macd_sig)
    col_macd = [f'macd_{col}' for col in prices]
    col_signal = [f'macd_sig_{col}' for col in prices]
    col_hist = [f'macd_hist_{col}' for col in prices]

    df.loc[:, col_macd] = macd
    df.loc[:, col_signal] = signal
    df.loc[:, col_hist] = histogram

    # fast Bollinger Bands
    bband_upper_values, bband_mid_values, bband_lower_values = calculate_bband_jit(values, window=lag_bband, num_std_dev=2)
    col_bband_upper = [f'bband_upper_{col}' for col in prices]
    col_bband_mid = [f'bband_mid_{col}' for col in prices]
    col_bband_lower = [f'bband_lower_{col}' for col in prices]

    df.loc[:, col_bband_upper] = bband_upper_values
    df.loc[:, col_bband_mid] = bband_mid_values
    df.loc[:, col_bband_lower] = bband_lower_values
        
    if lag_type == 'intraday':
        df.loc[df['seconds_in_bucket'] < lag_rsi * 10, col_rsi] = np.nan
        df.loc[df['seconds_in_bucket'] < lag_macd_long * 10, col_macd + col_signal + col_hist] = np.nan
        df.loc[df['seconds_in_bucket'] < lag_bband * 10, col_bband_upper + col_bband_mid + col_bband_lower] = np.nan
    return df

### All Features Compilation

In [18]:
def handlenan(df, test=False):
    columns_to_drop = ['target']
    columns_to_fill = list(set(df.columns) - set(columns_to_drop))
    if not test:
        df.dropna(subset=columns_to_drop, inplace=True)
        df.fillna(0, inplace=True)
    else:
        df[columns_to_fill] = df[columns_to_fill].fillna(0)
    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.reset_index(drop=True)
    return df

In [19]:
# 🚀 Function to generate all features by combining imbalance and other features
def generate_all_features(df, test=False, log=False):
    # Select relevant columns for feature generation
    df[['far_price', 'near_price']] = df[['far_price', 'near_price']].fillna(0)
    df['imbalance_size_signed'] = df['imbalance_size'] * df['imbalance_buy_sell_flag']
    
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "imbalance_buy_sell_flag"]]
    df = df[cols]
    
    # Generate TA features
    if log:
        print('Generating TA features...')
        start = time.time()
    df = generate_ta(df, lag_type='nolimit')
#     if not test:
    df = reduce_mem_usage(df)
    if log:
        print('Duration: ', (time.time() - start), '[s]\n')
    
    # Generate imbalance and other features
    if log:
        print('Generating Imbalance features...')
        start = time.time()
    df = imbalance_features(df, log)
    if not test:
        df = reduce_mem_usage(df)
    if log:
        print('Duration: ', time.time() - start, '[s]\n')
        print('Generating Other features...')
        start = time.time()
#     df = other_features(df)
    if log:
        print('Duration: ', time.time() - start, '[s]')
    if not test:
        df = reduce_mem_usage(df)
    if log:
        print('Garbage collecteing...')   
    gc.collect()  # Perform garbage collection to free up memory
    if log:
        print('Nan handling...')
    df = handlenan(df, test)
    if log:
        print('done')
        
    df = reduce_mem_usage(df)
    # Select and return the generated features
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id", "imbalance_size"]]
    
    return df[feature_name], df[["target", "date_id"]]

## Data Splitting

In [20]:
# Check if the code is running in offline or online mode
if is_offline:
    # In offline mode, split the data into training and validation sets based on the split_day
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    
    # Display a message indicating offline mode and the shapes of the training and validation sets
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
else:
    # In online mode, use the entire dataset for training
    df_train = df
    print("Online mode")
del df

Online mode


In [21]:
if is_train:
    print("Filling missed rows...")
    df_train = fill_missed_rows(df_train)
    if is_offline:
        X_train, y_train = generate_all_features(df_train, log=True)
        print("Build Train Feats Finished.")
        df_valid = fill_missed_rows(df_valid)
        X_valid, y_valid = generate_all_features(df_valid, log=True)
        print("Build Valid Feats Finished.")
        X_valid = reduce_mem_usage(X_valid)
        date_ids_valid = y_valid['date_id'].values
        y_valid = y_valid['target']
        del df_valid
    else:
        X_train, y_train = generate_all_features(df_train, log=True)
        print("Build Online Train Feats Finished.")
    del df_train
        
    X_train = reduce_mem_usage(X_train)
    date_ids = y_train['date_id'].values
    y_train = y_train['target']
    gc.collect()
    global_X_shape = X_train.shape
    print(X_train.shape, y_train.shape)


## LightGBM

In [22]:
feats_num = 256
model_load_dir = f'/kaggle/input/lgbm-{feats_num}feats-mean/'
model_save_path = f'lgbm-{feats_num}feats-mean'  # Directory to save models
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

models = [] 

In [23]:
""" 5-fold lgbm """
num_folds = 10
fold_size = 480 // num_folds
gap = 2

if is_train:
    scores = []

    for i in range(num_folds-5):
        start = i * fold_size
        end = start + fold_size

        # Define the purged set ranges
        purged_before_start = start - gap
        purged_before_end = start + gap
        purged_after_start = end - gap
        purged_after_end = end + gap

        # Exclude the purged ranges from the test set
        purged_set = ((date_ids >= purged_before_start) & (date_ids <= purged_before_end)) | \
                     ((date_ids >= purged_after_start) & (date_ids <= purged_after_end))

        # Define test_indices excluding the purged set
        test_indices = (date_ids >= start) & (date_ids < end) & ~purged_set
        train_indices = ~test_indices & ~purged_set

        X_fold_train = X_train[train_indices]
        y_fold_train = y_train[train_indices]
        X_fold_valid = X_train[test_indices]
        y_fold_valid = y_train[test_indices]

        print(f"Fold {i+1} Model Training")

        # Set up parameters for LightGBM
        lgb_params = {
                "random_state": 42*i,
                "objective": "mae",
                "n_estimators": 4000,
                "num_leaves": 256,
                "subsample": 0.6,
                "colsample_bytree": 0.8,
                "learning_rate": 0.01,
                'max_depth': 11,
                "n_jobs": 4,
                "device": "gpu",
                "verbosity": -1,
                "importance_type": "gain",
                "reg_alpha": 0.2,
                "reg_lambda": 3.25
            }
        
        # Train a LightGBM model for the current fold
        lgb_model = lgb.LGBMRegressor(**lgb_params)
        lgb_model.fit(
            X_fold_train,
            y_fold_train,
            eval_set=[(X_fold_valid, y_fold_valid)],
            callbacks=[
                lgb.callback.early_stopping(stopping_rounds=100),
                lgb.callback.log_evaluation(period=100),
            ],
        )

        # Append the model to the list
        models.append(lgb_model)
        # Save the model to a file
        model_filename = os.path.join(model_save_path, f'model_10folds_{i+1}.txt')
        lgb_model.booster_.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        fold_predictions = lgb_model.predict(X_fold_valid)
        fold_score = mean_absolute_error(fold_predictions, y_fold_valid)
        scores.append(fold_score)
        print(f"Fold {i+1} MAE: {fold_score}")

        # Free up memory by deleting fold specific variables
        del X_fold_train, y_fold_train, X_fold_valid, y_fold_valid
        gc.collect()

In [24]:
if is_train and is_offline:
    for seed in [42]:
        print(f"Seed = {seed}")
        # Set up parameters for LightGBM
        lgb_params = {
                "random_state": seed,
                "objective": "mae",
                "n_estimators": 4000,
                "num_leaves": 256,
                "subsample": 0.6,
                "colsample_bytree": 0.8,
                "learning_rate": 0.01,
                'max_depth': 11,
                "n_jobs": 4,
                "device": "gpu",
                "verbosity": -1,
                "importance_type": "gain",
                "reg_alpha": 0.2,
                "reg_lambda": 3.25
            }

        # Train the model on the entire dataset
        model = lgb.LGBMRegressor(**lgb_params)

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[
                lgb.callback.early_stopping(stopping_rounds=100),
                lgb.callback.log_evaluation(period=100),
            ],
        )

        # Append the final model to the list of models
        models.append(model)

        # Save the final model to a file
        model_filename = os.path.join(model_save_path, f'model_offline_seed{seed}_midsec.txt')
        model.booster_.save_model(model_filename)
        print(f"Final model saved to {model_filename}")
        
    for i, model in enumerate(models):
        X_valid['preds'] = model.predict(X_valid)
        X_valid['date_id'] = date_ids_valid
        for group_id, values in X_valid.groupby(['date_id', 'seconds_in_bucket'])['preds']:
            X_valid.loc[values.index, 'preds'] -= np.mean(values)
        print(f"model_{i}, MAE: {mean_absolute_error(y_valid, X_valid['preds'])}")
print(f"{len(models)} models are ready")

0 models are ready


In [25]:
seed = 42
if not is_offline:
    if is_train:
        print(f"Seed = {seed}")
        # Set up parameters for LightGBM
        lgb_params = {
                "random_state": seed,
                "objective": "mae",
                "n_estimators": 3000,
                "num_leaves": 256,
                "subsample": 0.6,
                "colsample_bytree": 0.8,
                "learning_rate": 0.01,
                'max_depth': 11,
                "n_jobs": 4,
                "device": "gpu",
                "verbosity": -1,
                "importance_type": "gain",
                "reg_alpha": 0.2,
                "reg_lambda": 3.25
            }

        # Train the model on the entire dataset
        model = lgb.LGBMRegressor(**lgb_params)

        model.fit(
            X_train,
            y_train,
            callbacks=[
                lgb.callback.log_evaluation(period=100),
            ],
            # init_model=init_model
        )

        # Append the final model to the list of models
        models.append(model)

        # Save the final model to a file
        model_filename = os.path.join(model_save_path, f'model_seed{seed}.txt')
        model.booster_.save_model(model_filename)
        print(f"Final model saved to {model_filename}")

    else:  # if is not train (use saved models)
        for i in range(5):
            model_load_file = os.path.join(model_load_dir, f'model_10folds_{i+1}.txt')
            models.append(lgb.Booster(model_file=model_load_file))
        model_load_file = os.path.join(model_load_dir, f'model_seed{seed}.txt')
        models.append(lgb.Booster(model_file=model_load_file))
    print(f"{len(models)} models are ready")

6 models are ready


# Inference

In [26]:
online_train_decay = 1.

def revealed_to_filled_df(revealed):
    df = pd.DataFrame({'date_id':revealed['revealed_date_id'],
                       'stock_id':revealed['stock_id'],
                       'seconds_in_bucket':revealed['seconds_in_bucket'],
                       'target':revealed['revealed_target']})
    df['row_id'] = [f'{d}_{t}_{s}' for d, t, s in 
                    zip(df['date_id'],df['seconds_in_bucket'],df['stock_id'])]
    return fill_missed_rows(df)


def cut_tail(cache, days_to_cache):
    cache_size = days_to_cache * STOCK_NUM * SEC_NUM
    if len(cache) < cache_size:
        return cache
    cache = cache.tail(cache_size).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id'])
    return cache.reset_index(drop=True)

    
def sqrt_mean(preds_list):
    preds = np.vstack(preds_list)
    preds_sgn = np.sign(preds)
    preds = np.sqrt(np.abs(preds))
    preds = preds * preds_sgn
    pred = np.mean(preds, axis=0)
    pred_sgn = np.sign(pred)
    pred = np.square(pred)
    pred = pred * pred_sgn
    return pred


if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    y_min, y_max = -64, 64
    qps, gen_time,inf_time = [], [], []
    cache = pd.DataFrame()
    
    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        
        # pad rows for missed stocks
        test['not_pad'] = [1]*len(test)
        test = test.drop(['currently_scored'], axis=1)
        test_filled = fill_missed_rows(test, mode='per_second')
        
        # save revealed targets of previous date
        cur_sec = test['seconds_in_bucket'].values[0]
        if cur_sec == 0:
            revealed_df = revealed_to_filled_df(revealed_targets)
            if counter == 0:
                cache = revealed_df
            else:
                cache = fill_missed_rows(cache)  # in case of missed seconds in the previous day
                cache = cut_tail(cache, days_to_cache=5)
                cache.loc[cache.index[-STOCK_NUM*SEC_NUM:], 'target'] = revealed_df['target'].values
                cache = pd.concat([cache, test_filled], ignore_index=True, axis=0)
        cache = pd.concat([cache, test_filled], ignore_index=True, axis=0)
        gen_start = time.time()
        feat, target = generate_all_features(cache, test=True, log=False)
        gen_time.append(time.time() - gen_start)
        
        if online_train_decay < 1.:
            if cur_sec == 0 and counter != 0:
                start = time.time()
                feats_refit = feat[-STOCK_NUM*SEC_NUM-len(test_filled):-len(test_filled)].drop('not_pad', axis=1)
                target_refit = target[-STOCK_NUM*SEC_NUM-len(test_filled):-len(test_filled)]
                for i in range(len(models)):
                    models[i] = models[i].refit(feats_refit, target_refit['target'], decay_rate=online_train_decay)
                print(f"refit time = {time.time() - start}")
        
        feat = feat[-len(test_filled):]
        feat = feat[feat['not_pad'] == 1].drop('not_pad', axis=1)
        
        # Generate predictions for each model and calculate the weighted average
        inf_start = time.time()
        preds_list = []
        for model in models:
            preds_list.append(model.predict(feat))
        predictions = sum(preds_list) / len(preds_list)  
        predictions = predictions - np.mean(predictions)
        predictions = np.clip(predictions, y_min, y_max)
        sample_prediction['target'] = predictions
        inf_time.append(time.time() - inf_start)
        env.predict(sample_prediction)
        
        counter += 1
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps), 'gen_time', np.mean(gen_time), 'inf_time', np.mean(inf_time))

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
10 qps: 1.1259600400924683 gen_time 0.8338542222976685 inf_time 0.2704096794128418
20 qps: 1.1199040174484254 gen_time 0.8176470041275025 inf_time 0.28276400566101073
30 qps: 1.1096340894699097 gen_time 0.8121590296427409 inf_time 0.27860652605692543
40 qps: 1.1073296129703523 gen_time 0.8116705775260925 inf_time 0.2771796226501465
50 qps: 1.116520504951477 gen_time 0.8154175806045533 inf_time 0.28273979663848875
60 qps: 1.1320046504338583 gen_time 0.8248473962148031 inf_time 0.28766753276189166
70 qps: 1.1481600965772356 gen_time 0.8372050421578544 inf_time 0.29161362988608225
80 qps: 1.1591553866863251 gen_time 0.8505742192268372 inf_time 0.2893630117177963
90 qps: 1.1673168314827813 gen_time 0.8605035543441772 inf_time 0.2877385007010566
100 qps: 1.1852417945861817 gen_time 0.8727592468261719 inf_time 0.29342265367507936
110 qps: 1.2088516322049228 gen_time 0.